In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib2, json, argparse
import logging, os, io, requests
import key # import api key
from pprint import pprint
from jsonmerge import merge
import glob
import itertools
from bs4 import BeautifulSoup

# force utf8 encoding
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

# make plots show up inline
%matplotlib inline 

In [2]:
api_key = key.API_KEY
base_url = 'http://www.giantbomb.com/api'

In [3]:
offset=0
payload = {'api_key': api_key, 'format':'json', 'offset':offset, 'limit':1}

In [4]:
offsets = [0, 101, 201, 301, 401, 501, 601]
gb = []
for x in offsets:
    payload = {'api_key': api_key, 'format':'json', 'offset':x}
    results = requests.get(base_url + "/reviews/", params=payload).json()['results']
    gb.append(results)
    # because giantbomb limits us to 100 records at a time,
    # save each record as a separate file
    with io.open('%s.json' % x, 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(results, ensure_ascii=False)))

In [5]:
result = []
for offset in offsets:
    with open(str(offset) + '.json') as data_file:
        result.append(json.load(data_file))

In [6]:
# use itertools to extract double bracketed list
merged_list = list(itertools.chain(*result))

df = pd.DataFrame(merged_list)

In [21]:
# create a new collumn
df['description_strip'] = np.nan
# strip html from description field
for index, row in df.iterrows():
    html = row['description']
    soup = BeautifulSoup(html)
    text = soup.getText()
    df['description_strip'][index] = text

In [10]:
game = df['game']
game = game.apply(pd.Series)
game.drop('api_detail_url', axis=1, inplace=True)

In [11]:
game.head(5)

,id,name,site_detail_url
0,20462,SingStar,http://www.giantbomb.com/singstar/3030-20462/
1,20455,Haze,http://www.giantbomb.com/haze/3030-20455/
2,20669,Metal Gear Solid 4: Guns of the Patriots,http://www.giantbomb.com/metal-gear-solid-4-gu...
3,20681,Poker Smash,http://www.giantbomb.com/poker-smash/3030-20681/
4,20655,Battlefield: Bad Company,http://www.giantbomb.com/battlefield-bad-compa...


In [12]:
merged = pd.concat([df, game], axis=1)
merged['site'] = 'Giant Bomb'

In [13]:
giantbomb = merged[['name', 
                      'publish_date',  
                      'reviewer',
                      'api_detail_url',
                      'description_strip',
                      'site',
                      'platforms',
                      'score' 
                      ]]

In [14]:
giantbomb.columns = ['game', 'date', 'reviewer', 
                     'link', 'review', 'site', 
                     'platforms', 'score']

In [15]:
# get names of reivewers
pd.Series(giantbomb['reviewer'].values.ravel()).unique()

array([u'Jeff', u'Brad', u'Ryan', u'Vinny', u'Andy', u'Dave', u'Drew',
       u'Alex', u'Matt', u'Patrick', u'Dan'], dtype=object)

In [16]:
# Full names for Reviewers
giantbomb.ix[giantbomb.reviewer == "Jeff", 'reviewer'] = 'Jeff Gerstmann'
giantbomb.ix[giantbomb.reviewer == "Brad", 'reviewer'] = 'Brad Shoemaker'
giantbomb.ix[giantbomb.reviewer == "Ryan", 'reviewer'] = 'Ryan Davis'
giantbomb.ix[giantbomb.reviewer == "Vinny", 'reviewer'] = 'Vinny Caravella'
giantbomb.ix[giantbomb.reviewer == "Andy", 'reviewer'] = 'Andy McCurdy'
giantbomb.ix[giantbomb.reviewer == "Dave", 'reviewer'] = 'Dave Lang'
giantbomb.ix[giantbomb.reviewer == "Drew", 'reviewer'] = 'Drew Scanlon'
giantbomb.ix[giantbomb.reviewer == "Alex", 'reviewer'] = 'Alex Navarro'
giantbomb.ix[giantbomb.reviewer == "Matt", 'reviewer'] = 'Matthew Rorie'
giantbomb.ix[giantbomb.reviewer == "Patrick", 'reviewer'] = 'Patrick Klepek'
giantbomb.ix[giantbomb.reviewer == "Dan", 'reviewer'] = 'Dan Ryckert'

In [17]:
giantbomb['score_100'] = giantbomb['score']*20

In [140]:
giantbomb.to_csv('giantbomb_reviews.csv', encoding='utf-8', index=False)

In [18]:
giantbomb

,game,date,reviewer,link,review,site,platforms,score,score_100
0,SingStar,2008-05-23 13:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-1/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,PlayStation 3,4,80
1,Haze,2008-05-20 17:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-2/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,PlayStation 3,2,40
2,Metal Gear Solid 4: Guns of the Patriots,2008-06-20 12:00:00,Brad Shoemaker,http://www.giantbomb.com/api/review/1900-3/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,PlayStation 3,5,100
3,Poker Smash,2008-02-29 12:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-4/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,Xbox 360 Games Store,4,80
4,Battlefield: Bad Company,2008-07-06 12:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-5/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,Xbox 360,5,100
5,LEGO Indiana Jones: The Original Adventures,2008-07-02 12:00:00,Brad Shoemaker,http://www.giantbomb.com/api/review/1900-6/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,"PlayStation 3, Xbox 360",3,60
6,Robert Ludlum's The Bourne Conspiracy,2008-07-02 12:00:00,Ryan Davis,http://www.giantbomb.com/api/review/1900-7/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,"Xbox 360, PlayStation 3",3,60
7,Burnout Paradise,2008-03-01 12:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-8/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,"PlayStation 3, Xbox 360",5,100
8,Professor Layton and the Curious Village,2008-03-07 10:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-9/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,Nintendo DS,5,100
9,Super Smash Bros. Brawl,2008-03-13 15:00:00,Jeff Gerstmann,http://www.giantbomb.com/api/review/1900-10/,"2011's Mortal Kombat was a terrific, fresh st...",Giant Bomb,Wii,4,80
